# Mock QCNN Training

In [ ]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from qiskit import IBMQ
from qiskit.providers.fake_provider import FakeJakarta, FakeMontreal
from qiskit_aer.noise import NoiseModel


noisy = FakeJakarta()
noise_model = NoiseModel.from_backend(noisy)
coupling_map = noisy.configuration().coupling_map
basis_gates = noise_model.basis_gates

dev = qml.device(
    'qiskit.aer',
    wires=4,
    shots=1024,
    noise_model=noise_model,
    coupling_map=coupling_map,
    basis_gates=basis_gates
)

PATH_Model2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Model2.pt'
PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Model3.pt'


# Mock QCNN

Hyperparameters

In [ ]:
steps = 100
learning_rate = 0.1
batch_size = 10
ansatz = 'TTN'

QCNN

In [ ]:
class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()


class x_transform3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 7 * 7)
        x = self.fc(x)
        return x.detach().numpy()

model2 = x_transform2()
model3 = x_transform3()

In [ ]:
def statepreparation(x, Trained):
    if Trained == False:
        embedding.Noisy_Four_QuantumEmbedding1(x)
    elif Trained == 'Model2':
        model2.load_state_dict(torch.load(PATH_Model2))
        x = model2(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x)
    elif Trained == 'Model3':
        model3.load_state_dict(torch.load(PATH_Model3))
        x = model3(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x[0])
    

@qml.qnode(dev)
def Noisy_QCNN_classifier(params, x, Trained):
    param1 = params[0:2]
    param2 = params[2:4]

    statepreparation(x, Trained)
    embedding.U_TTN(param1, wires=[0,1])
    embedding.U_TTN(param1, wires=[3,2])
    embedding.U_TTN(param2, wires=[1,2])
    return qml.expval(qml.PauliZ(2))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)


def cost(weights, X_batch, Y_batch, Trained):
    preds = [Noisy_QCNN_classifier(weights, x, Trained) for x in X_batch]
    return Linear_Loss(Y_batch, preds)


def circuit_training(X_train, Y_train, Trained):

    weights = np.random.random(4, requires_grad = True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, Trained),
                                                     weights)
        loss_history.append(cost_new)
        print("iteration: ", it, " cost: ", cost_new)
    return loss_history, weights

# Model 2

In [ ]:

feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:

Loss_histories_not_trained, weights_not_trained, Loss_histories_Model2, weights_Model2 = [], [], [], []

for i in range(3):
    loss_not_trained, weight_not_trained = circuit_training(X_train, Y_train, False)
    loss_Model2, weight_Model2 = circuit_training(X_train, Y_train, 'Model2')

    Loss_histories_not_trained.append(loss_not_trained)
    Loss_histories_Model2.append(loss_Model2)

    weights_not_trained.append(weight_not_trained)
    weights_Model2.append(weight_Model2)

Loss_histories_not_traiend, Loss_histories_Model2 =  np.array(Loss_histories_not_trained), np.array(Loss_histories_Model2)


f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Loss_not_trained.txt', 'a')
f.write(str(Loss_histories_not_trained))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/weights_not_trained.txt', 'a')
f.write(str(weights_not_trained))
f.close()


f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Loss_Model2.txt', 'a')
f.write(str(Loss_histories_Model2))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/weights_Model2.txt', 'a')
f.write(str(weights_Model2))
f.close()

# Model3

## 0. Getting Started

In [ ]:
feature_reduction = False
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

In [ ]:
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
Loss_histories_Model3, weights_Model3 = [], []
for i in range(3):

    loss_Model3, weight_Model3 = circuit_training(X_train, Y_train, 'Model3')

    Loss_histories_Model3.append(loss_Model3)
    weights_Model3.append(weight_Model3)

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Loss_Model3.txt', 'a')
f.write(str(Loss_histories_Model3))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/weights_Model3.txt', 'a')
f.write(str(weights_Model3))
f.close()